In [ ]:
from pyspark.sql.types import *
import pyspark.sql.functions as sf

fødsel = spark.createDataFrame([], StructType([
    StructField('personidentifikator',StringType()),
    StructField('foedselsaar', StringType())]))

skatt = spark.createDataFrame([], StructType([
    StructField('skatt', StructType([
        StructField('personidentifikator',StringType()),
        StructField('konto', StructType([
            StructField('innskudd', IntegerType())]))
        ]))
    ])
)

fødsel.printSchema()
skatt.printSchema()

root
 |-- personidentifikator: string (nullable = true)
 |-- foedselsaar: string (nullable = true)

root
 |-- skatt: struct (nullable = true)
 |    |-- personidentifikator: string (nullable = true)
 |    |-- konto: struct (nullable = true)
 |    |    |-- innskudd: integer (nullable = true)



In [ ]:
innskudd_for_alder = skatt\
    .select("skatt.personidentifikator", "skatt.konto.innskudd")\
    .groupBy("personidentifikator")\
    .agg(sf.sum("innskudd").alias("sum_innskudd"))
    
innskudd_for_alder.printSchema()

root
 |-- personidentifikator: string (nullable = true)
 |-- sum_innskudd: long (nullable = true)



In [ ]:
innskudd_for_alder.printLineageTemplate([
    {"/input/fødsel": fødsel },
    {"/input/konto": konto }
])

{
  "lineage" : {
    "name" : "spark_schema",
    "type" : "structure",
    "fields" : [ {
      "name" : "personidentifikator",
      "type" : "inherited",
      "confidence" : 0.9,
      "sources" : [ {
        "field" : "personidentifikator",
        "path" : "/input/fødsel",
        "version" : 123
      }, {
        "field" : "skatt.personidentifikator",
        "path" : "/input/konto",
        "version" : 123
      } ]
    }, {
      "name" : "sum_innskudd",
      "type" : "derived/created",
      "confidence" : 0.0
    } ]
  }
}
